In [26]:
import cv2
import numpy as np

# 이전 프레임의 차선 위치
prev_lane_position = None

def estimate_speed(frame, prev_lane_position):
    # 현재 프레임에서 차선을 감지 또는 추적합니다.
    current_lane_position = detect_lane(frame)
    
    # 이전 프레임과 현재 프레임의 차선 위치를 사용하여 속도를 계산합니다.
    if prev_lane_position is not None and current_lane_position is not None:
        # 이전 프레임과 현재 프레임의 차선 중심 간의 거리를 계산합니다.
        distance_px = abs(current_lane_position - prev_lane_position)
        
        # 픽셀 단위 거리를 실제 거리로 변환합니다. (픽셀 단위 거리 / 픽셀 단위 길이)
        # 예를 들어, 1 픽셀 단위 거리는 0.1미터가 될 수 있습니다.
        distance_m = distance_px * 0.005  # 예시: 1 픽셀 단위 거리는 0.1 미터라고 가정
        
        # 초당 이동 거리를 계산합니다.
        speed_mps = distance_m * fps  # 초당 이동 거리 = 픽셀 단위 거리 * 초당 프레임 수
        # m/s 를 km/h 로 변환
        speed_kph = speed_mps * 3.6
        
        return speed_kph
    
    else:
        return None

def detect_lane(frame):
    # 이미지를 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 가우시안 블러를 적용하여 노이즈 제거
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Canny 엣지 감지를 사용하여 엣지 검출
    edges = cv2.Canny(blurred, 50, 150)
    
    # ROI(Region of Interest)를 정의하여 차선 영역에 대한 관심 영역 설정
    height, width = frame.shape[:2]
    roi_vertices = [
        (0, height),
        (width / 2, height / 2),
        (width, height)
    ]
    mask = np.zeros_like(edges)
    cv2.fillPoly(mask, [np.array(roi_vertices, dtype=np.int32)], 255)
    masked_edges = cv2.bitwise_and(edges, mask)
    
    # 허프 변환을 사용하여 선분을 검출
    lines = cv2.HoughLinesP(masked_edges, 1, np.pi / 180, 50, minLineLength=50, maxLineGap=100)
    
    # 검출된 선분들을 원본 이미지에 그립니다.
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
    
    # 검출된 차선의 중앙을 계산하여 반환합니다.
    if lines is not None and len(lines) > 0:
        lane_position = np.mean([line[0][0] for line in lines])
        return lane_position
    else:
        return None

# 비디오 파일 열기
cap = cv2.VideoCapture('/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV')

# 프레임 수 계산
fps = cap.get(cv2.CAP_PROP_FPS)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 속도 추정
    estimated_speed = estimate_speed(frame, prev_lane_position)
    
    if estimated_speed is not None:
        if estimated_speed < 10:  # 예시: 속도가 5km/h 미만이면 정지 상태로 간주
            driving_status = "STOP"
        else:
            driving_status = "DRIVING"
        print(f"Estimated speed: {estimated_speed} km/h, Driving status: {driving_status}")
        
        # 영상에 주행 상태 표시
        cv2.putText(frame, f"Status: {driving_status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    # 이전 프레임의 차선 위치를 업데이트
    prev_lane_position = detect_lane(frame)
    
    # 영상 출력
    cv2.imshow('Lane Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 비디오 파일 닫기
cap.release()
cv2.destroyAllWindows()


Estimated speed: 19.03026954177894 km/h, Driving status: DRIVING
Estimated speed: 44.5006168831169 km/h, Driving status: DRIVING
Estimated speed: 34.13578947368425 km/h, Driving status: DRIVING
Estimated speed: 42.77761124121785 km/h, Driving status: DRIVING
Estimated speed: 5.593965517241372 km/h, Driving status: STOP
Estimated speed: 81.46708474576272 km/h, Driving status: DRIVING
Estimated speed: 47.588038277511956 km/h, Driving status: DRIVING
Estimated speed: 43.96999068033547 km/h, Driving status: DRIVING
Estimated speed: 35.35440823327617 km/h, Driving status: DRIVING
Estimated speed: 57.520746928746966 km/h, Driving status: DRIVING
Estimated speed: 30.377213114754074 km/h, Driving status: DRIVING
Estimated speed: 25.01620608899297 km/h, Driving status: DRIVING
Estimated speed: 36.87451127819551 km/h, Driving status: DRIVING
Estimated speed: 19.40211290322584 km/h, Driving status: DRIVING
Estimated speed: 36.43788610871438 km/h, Driving status: DRIVING
Estimated speed: 22.545974

In [25]:
import cv2

# 배경 제거 모델 초기화
fgbg = cv2.createBackgroundSubtractorMOG2()

def estimate_driving_status(frame):
    # 배경 제거 적용
    fgmask = fgbg.apply(frame)
    
    # 배경만 남기고 나머지는 제거
    background_only = cv2.bitwise_and(frame, frame, mask=fgmask)

    # 프레임을 그레이스케일로 변환합니다.
    gray = cv2.cvtColor(background_only, cv2.COLOR_BGR2GRAY)
    
 
    prev_frame = gray  #이전 프레임을 현재 프레임으로 설정
    
    # 현재 프레임과 이전 프레임 간의 차이를 계산합니다.
    frame_diff = cv2.absdiff(prev_frame, gray)
    
    # 차량의 움직임이 있는지 여부를 확인합니다.
    motion_detected = False
   
    
    # 주행 중인지 여부를 판단합니다.
    if motion_detected:
        driving_status = "DRIVING"
    else:
        driving_status = "STOP"
    
    return driving_status

# 비디오 파일 열기
cap = cv2.VideoCapture('/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # 주행 상태 추정
    status = estimate_driving_status(frame)
    
    # 주행 상태 표시
    cv2.putText(frame, f"Status: {status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    
    # 영상 출력
    cv2.imshow('Driving Status Estimation', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 비디오 파일 닫기
cap.release()
cv2.destroyAllWindows()


In [28]:
import cv2
from ultralytics import YOLO
from collections import deque, Counter

class TrafficLightDetector:
    def __init__(self, model_path, history_length=30):
        self.model = YOLO(model_path)
        self.desired_width = 640
        self.desired_height = 640
        self.history_length = history_length
        self.size_history = deque(maxlen=history_length)

    def detect_traffic_light(self, frame):
        frame = cv2.resize(frame, (self.desired_width, self.desired_height))
        results = self.model.track(frame, persist=True)
        detected_objects = []

        if len(results) > 0:
            for detection in results[0].boxes.data:
                x1, y1, x2, y2 = detection[:4]
                size = (x2 - x1) * (y2 - y1)  # 신호등 크기 계산
                detected_objects.append(size)

        # 업데이트된 크기 히스토리
        self.size_history.extend(detected_objects)
        
        # 신호등 크기 변화를 통한 주행 상태 판단
        is_driving = self.check_driving_status()
        return is_driving, results

    def check_driving_status(self):
        # 신호등 크기의 변화를 확인하여 주행 상태 판단
        if len(self.size_history) == self.history_length:
            # 최근 크기 변화 계산
            recent_changes = max(self.size_history) - min(self.size_history)
            # 주행 상태 판단: 크기 변화가 있는 경우 주행 중으로 판단
            if recent_changes > threshold:
                return True
        # 주행 상태 판단: 크기 변화가 없는 경우 정지 중으로 판단
        return False

def main(video_path, model_path):
    cap = cv2.VideoCapture(video_path)
    detector = TrafficLightDetector(model_path)
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        is_driving, results = detector.detect_traffic_light(frame)
        if is_driving:
            status = "DRIVING"
        else:
            status = "STOP"
        # 결과 출력
        cv2.putText(frame, f"Status: {status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        if len(results) > 0:
            annotated_frame = results[0].plot()
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    model_path = "/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/traffic_light/traffic_best_ver2.pt"
    threshold = 100  # 주행 상태 판단을 위한 임계값 설정
    main(video_path, model_path)



0: 640x640 (no detections), 9.4ms
Speed: 2.5ms preprocess, 9.4ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.6ms
Speed: 1.5ms preprocess, 7.6ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.4ms
Speed: 2.1ms preprocess, 9.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.2ms
Speed: 3.1ms preprocess, 12.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.2ms
Speed: 1.7ms preprocess, 7.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.4ms
Speed: 2.7ms preprocess, 8.4ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.3ms
Speed: 3.4ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.0ms
Speed: 1.7ms preprocess, 12.0ms inference, 0.

In [1]:
import cv2
from ultralytics import YOLO
from collections import deque, Counter

class TrafficLightDetector:
    def __init__(self, model_path, history_length=30):
        self.model = YOLO(model_path)
        self.desired_width = 640
        self.desired_height = 640
        self.history_length = history_length
        self.size_history = deque(maxlen=history_length)
        self.signal_detected = False

    def detect_traffic_light(self, frame):
        frame = cv2.resize(frame, (self.desired_width, self.desired_height))
        results = self.model.track(frame, persist=True)
        detected_objects = []

        if len(results) > 0:
            self.signal_detected = True
            for detection in results[0].boxes.data:
                x1, y1, x2, y2 = detection[:4]
                size = (x2 - x1) * (y2 - y1)  # 신호등 크기 계산
                detected_objects.append(size)
        else:
            self.signal_detected = False

        # 업데이트된 크기 히스토리
        self.size_history.extend(detected_objects)
        
        # 신호등이 인식되었을 때 주행 상태 판단
        if self.signal_detected:
            is_driving = self.check_driving_status()
            return is_driving, results
        else:
            return None, results

    def check_driving_status(self):
        # 신호등 크기의 변화를 확인하여 주행 상태 판단
        if len(self.size_history) == self.history_length:
            # 최근 크기 변화 계산
            recent_changes = max(self.size_history) - min(self.size_history)
            # 주행 상태 판단: 크기 변화가 있는 경우 주행 중으로 판단
            if recent_changes > threshold:
                return True
        # 주행 상태 판단: 크기 변화가 없는 경우 정지 중으로 판단
        return False

def main(video_path, model_path):
    cap = cv2.VideoCapture(video_path)
    detector = TrafficLightDetector(model_path)
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        is_driving, results = detector.detect_traffic_light(frame)
        if is_driving is not None:
            if is_driving:
                status = "DRIVING"
            else:
                status = "STOP"
            # 결과 출력
            cv2.putText(frame, f"Status: {status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        elif not detector.signal_detected:
            # 신호등이 인식되지 않으면 주행 상태 표시를 제거
            pass
        if len(results) > 0:
            annotated_frame = results[0].plot()
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    model_path = "/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/traffic_light/traffic_best_ver2.pt"
    threshold = 100  # 주행 상태 판단을 위한 임계값 설정
    main(video_path, model_path)



0: 640x640 (no detections), 7.4ms
Speed: 3.1ms preprocess, 7.4ms inference, 441.4ms postprocess per image at shape (1, 3, 640, 640)


QFactoryLoader::QFactoryLoader() checking directory path "/home/min/venv/yolo/lib/python3.10/site-packages/cv2/qt/plugins" ...
QFactoryLoader::QFactoryLoader() checking directory path "/usr/bin" ...
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/411toppm"
"Failed to extract plugin meta data from '/usr/bin/411toppm'" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7z"
QElfParser: '/usr/bin/7z' is not an ELF object (file too small)
"'/usr/bin/7z' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7za"
QElfParser: '/usr/bin/7za' is not an ELF object (file too small)
"'/usr/bin/7za' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr/bin/7zr"
QElfParser: '/usr/bin/7zr' is not an ELF object (file too small)
"'/usr/bin/7zr' is not an ELF object (file too small)" 
         not a plugin
QFactoryLoader::QFactoryLoader() looking at "/usr


0: 640x640 (no detections), 8.2ms
Speed: 4.5ms preprocess, 8.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.1ms
Speed: 3.9ms preprocess, 9.1ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.8ms
Speed: 2.3ms preprocess, 8.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.4ms
Speed: 5.4ms preprocess, 10.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 11.1ms
Speed: 5.6ms preprocess, 11.1ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.4ms
Speed: 5.5ms preprocess, 13.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 12.4ms
Speed: 8.0ms preprocess, 12.4ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.7ms
Speed: 4.2ms preprocess, 9.7ms inference

In [10]:
import cv2
from ultralytics import YOLO
from collections import deque, Counter

class TrafficLightDetector:
    def __init__(self, model_path, history_length=30):
        self.model = YOLO(model_path)
        self.desired_width = 640
        self.desired_height = 640
        self.history_length = history_length
        self.size_history = deque(maxlen=history_length)
        self.signal_detected = False

    def detect_traffic_light(self, frame):
        frame = cv2.resize(frame, (self.desired_width, self.desired_height))
        results = self.model.track(frame, persist=True)
        detected_objects = []

        if results[0].boxes.is_track:
            self.signal_detected = True
            for detection in results[0].boxes.data:
                x1, y1, x2, y2 = detection[:4]
                size = (x2 - x1) * (y2 - y1)  # 신호등 크기 계산
                detected_objects.append(size)
        else:
            self.signal_detected = False

        # 업데이트된 크기 히스토리
        self.size_history.extend(detected_objects)
        
        # 신호등이 인식되었을 때 주행 상태 판단
        if self.signal_detected:
            is_driving = self.check_driving_status()
            return is_driving, results
        else:
            return None, results

    def check_driving_status(self):
        # 신호등 크기의 변화를 확인하여 주행 상태 판단
        if len(self.size_history) == self.history_length:
            # 최근 크기 변화 계산
            recent_changes = max(self.size_history) - min(self.size_history)
            # 주행 상태 판단: 크기 변화가 있는 경우 주행 중으로 판단
            if recent_changes > threshold:
                return True
        # 주행 상태 판단: 크기 변화가 없는 경우 정지 중으로 판단
        return False

def main(video_path, model_path):
    cap = cv2.VideoCapture(video_path)
    detector = TrafficLightDetector(model_path)
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        is_driving, results = detector.detect_traffic_light(frame)
        if is_driving is not None:
            if is_driving:
                status = "DRIVING"
            else:
                status = "STOP"
            # 결과 출력
            cv2.putText(frame, f"Status: {status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        elif not detector.signal_detected:
            # 신호등이 인식되지 않으면 주행 상태 표시를 제거
            pass
        if len(results) > 0:
            annotated_frame = results[0].plot()
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    model_path = "/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/traffic_light/traffic_best_ver2.pt"
    threshold = 30  # 주행 상태 판단을 위한 임계값 설정
    main(video_path, model_path)



0: 640x640 (no detections), 7.5ms
Speed: 2.1ms preprocess, 7.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.5ms
Speed: 3.3ms preprocess, 7.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.8ms
Speed: 2.0ms preprocess, 7.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.0ms
Speed: 2.3ms preprocess, 8.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.1ms
Speed: 2.4ms preprocess, 7.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.9ms
Speed: 3.9ms preprocess, 8.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.3ms
Speed: 1.5ms preprocess, 8.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.8ms
Speed: 4.2ms preprocess, 9.8ms inference, 0.5ms 

최종!!!!

In [19]:
import cv2
from ultralytics import YOLO
from collections import deque, Counter

class TrafficLightDetector:
    def __init__(self, model_path, history_length=30):
        self.model = YOLO(model_path)
        self.desired_width = 640
        self.desired_height = 640
        self.history_length = history_length
        self.size_history = deque(maxlen=history_length)
        self.signal_detected = False
        self.signal_color = None
        self.classes = {
            0: 'Red',
            1: 'Red and Green Arrow',
            2: 'Green',
            3: 'Yellow',
            4: 'Green and Green Arrow',
            5: 'Green Arrow'
        }

    def detect_traffic_light(self, frame):
        frame = cv2.resize(frame, (self.desired_width, self.desired_height))
        results = self.model.track(frame, persist=True)
        detected_objects = []

        if results[0].boxes.is_track:
            self.signal_detected = True
            print(results[0].boxes)
            for detection in results[0].boxes.data:
                x1, y1, x2, y2 = detection[:4]
                size = (x2 - x1) * (y2 - y1)  # 신호등 크기 계산
                detected_objects.append(size)
                class_id = int(detection[6])
                if class_id == 0:
                    self.signal_color = "Red"
                elif class_id == 1:
                    self.signal_color = "Red and Green Arrow"
                elif class_id == 2:
                    self.signal_color = "Green"
                elif class_id == 3:
                    self.signal_color = "Yellow"
                elif class_id == 4:
                    self.signal_color = "Green and Green Arrow"
                elif class_id == 5:
                    self.signal_color = "Green Arrow"

        else:
            self.signal_detected = False

        # 업데이트된 크기 히스토리
        self.size_history.extend(detected_objects)
        
        # 신호등이 인식되었을 때 주행 상태 판단
        if self.signal_detected:
            is_driving = self.check_driving_status()
            return is_driving, results
        else:
            return None, results

    def check_driving_status(self):
        # 신호등 크기의 변화를 확인하여 주행 상태 판단
        if len(self.size_history) == self.history_length:
            # 최근 크기 변화 계산
            recent_changes = max(self.size_history) - min(self.size_history)
            # 주행 상태 판단: 크기 변화가 있는 경우 주행 중으로 판단
            if recent_changes > threshold:
                return True
        # 주행 상태 판단: 크기 변화가 없는 경우 정지 중으로 판단
        return False

def main(video_path, model_path):
    cap = cv2.VideoCapture(video_path)
    detector = TrafficLightDetector(model_path)
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        is_driving, results = detector.detect_traffic_light(frame)
        if is_driving is not None:
            if is_driving:
                status = "DRIVING"
            else:
                status = "STOP"
            # 결과 출력
            cv2.putText(frame, f"Status: {status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            if detector.signal_color is not None:
                # 신호등 색상 출력
                cv2.putText(frame, f"Signal color: {detector.signal_color}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        elif not detector.signal_detected:
            # 신호등이 인식되지 않으면 주행 상태 표시를 제거
            pass
        if len(results) > 0:
            annotated_frame = results[0].plot()
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    model_path = "/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/traffic_light/traffic_best_ver2.pt"
    threshold = 30  # 주행 상태 판단을 위한 임계값 설정
    main(video_path, model_path)



0: 640x640 (no detections), 8.0ms
Speed: 3.9ms preprocess, 8.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.9ms
Speed: 4.2ms preprocess, 8.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.5ms
Speed: 3.0ms preprocess, 8.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 8.3ms
Speed: 4.0ms preprocess, 8.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.3ms
Speed: 4.4ms preprocess, 9.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.7ms
Speed: 4.2ms preprocess, 10.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 13.4ms
Speed: 5.9ms preprocess, 13.4ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 14.0ms
Speed: 4.3ms preprocess, 14.0ms inference, 

pass fail 한번 줘볼까 pass->0 fail ->1

In [40]:
import cv2
from ultralytics import YOLO
from collections import deque, Counter

class TrafficLightDetector:
    def __init__(self, model_path, history_length=30):
        self.model = YOLO(model_path)
        self.desired_width = 640
        self.desired_height = 640
        self.history_length = history_length
        self.size_history = deque(maxlen=history_length)
        self.signal_detected = False
        self.signal_color = None
        self.classes = {
            0: 'Red',
            1: 'Red and Green Arrow',
            2: 'Green',
            3: 'Yellow',
            4: 'Green and Green Arrow',
            5: 'Green Arrow'
        }

    def detect_traffic_light(self, frame):
        frame = cv2.resize(frame, (self.desired_width, self.desired_height))
        results = self.model.track(frame, persist=True)
        detected_objects = []

        if results[0].boxes.is_track:
            self.signal_detected = True
            for detection in results[0].boxes.data:
                x1, y1, x2, y2 = detection[:4]
                size = (x2 - x1) * (y2 - y1)  # 신호등 크기 계산
                detected_objects.append(size)
                class_id = int(detection[6])
                if class_id == 0:
                    self.signal_color = "Red"
                elif class_id == 1:
                    self.signal_color = "Red and Green Arrow"
                elif class_id == 2:
                    self.signal_color = "Green"
                elif class_id == 3:
                    self.signal_color = "Yellow"
                elif class_id == 4:
                    self.signal_color = "Green and Green Arrow"
                elif class_id == 5:
                    self.signal_color = "Green Arrow"

        else:
            self.signal_detected = False

        # 업데이트된 크기 히스토리
        self.size_history.extend(detected_objects)
        
        # 신호등이 인식되었을 때'만' 주행 상태 판단
        if self.signal_detected:
            is_driving = self.check_driving_status()
            if is_driving and class_id in [0, 1]:
                Pass_Fail = "Fail"
            elif is_driving and class_id in [2, 3, 4, 5]:
                Pass_Fail = "Pass"
            elif not is_driving and class_id in [0, 1]:
                Pass_Fail = "Pass"
            elif not is_driving and class_id in [2, 3, 4, 5]:
                Pass_Fail = "Fail"
            return is_driving, results, Pass_Fail
        
        else:
            return None, results, None

    def check_driving_status(self): #주행 == true 주행  == false
        # 신호등 크기의 변화를 확인하여 주행 상태 판단
        if len(self.size_history) == self.history_length:
            # 최근 크기 변화 계산
            recent_changes = max(self.size_history) - min(self.size_history)
            # 주행 상태 판단: 크기 변화가 있는 경우 주행 중으로 판단
            if recent_changes > threshold:
                return True
        # 주행 상태 판단: 크기 변화가 없는 경우 정지 중으로 판단
        return False

def main(video_path, model_path):
    cap = cv2.VideoCapture(video_path)
    detector = TrafficLightDetector(model_path)
    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            break
        is_driving, results, pass_fail = detector.detect_traffic_light(frame)
        if is_driving is not None:
            if is_driving:
                status = "DRIVING"
            else:
                status = "STOP"
            # 결과 출력
            cv2.putText(frame, f"Status: {status}", (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            if detector.signal_color is not None:
                # 신호등 색상 출력
                cv2.putText(frame, f"Signal color: {detector.signal_color}", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                cv2.putText(frame, f"Pass/Fail: {pass_fail}", (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        elif not detector.signal_detected:
            # 신호등이 인식되지 않으면 주행 상태 표시를 제거
            pass
 
        
        if len(results) > 0:
            annotated_frame = results[0].plot()
            cv2.imshow("YOLOv8 Tracking", annotated_frame)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "/home/min/dev_ws/machine_learning/project_3rd/data/TEST_VIDEO/6_Fail_Pedestrian_Red_Light.MOV"
    model_path = "/home/min/dev_ws/machine_learning/project_3rd/src/model_learning/traffic_light/traffic_best_ver2.pt"
    threshold = 10  # 주행 상태 판단을 위한 임계값 설정
    main(video_path, model_path)


0: 640x640 (no detections), 7.2ms
Speed: 3.8ms preprocess, 7.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 11.4ms
Speed: 1.6ms preprocess, 11.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.7ms
Speed: 3.9ms preprocess, 10.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 7.5ms
Speed: 2.9ms preprocess, 7.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 10.8ms
Speed: 5.2ms preprocess, 10.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 11.1ms
Speed: 3.9ms preprocess, 11.1ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 14.5ms
Speed: 1.9ms preprocess, 14.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.3ms
Speed: 5.6ms preprocess, 9.3ms inferen